In [1]:
import os
import numpy as np
import cv2 
from PIL import Image

In [2]:
#set default parameters
base_dir = '...' #provide path to the parent folder
H = 512 #height of images
W = 512 #width of images
dermoscopic_image_folder = os.path.join(base_dir, 'dermoscopic_image')
hair_mask_folder = os.path.join(base_dir, 'hair_mask')
binarized_folder =  os.path.join(base_dir,'binarized')
transparent_folder =  os.path.join(base_dir,'transparent')
overlay_folder =  os.path.join(base_dir,'overlay')
collage_folder = os.path.join(base_dir,'collage')

In [3]:
#utility functions
def make_transparent(src_path, dst_path):
    """
    Makes the black part of a binary image fully transparent
    Arguments:
        src_path: input image path
        dst_pth: path to save transparent image
    """
    img = Image.open(src_path)
    img = img.convert("RGBA")
    datas = img.getdata()

    newData = []
    for item in datas:
        if item[0] == 0 and item[1] == 0 and item[2] == 0:
            newData.append((0, 0, 0, 0))
        else:
            newData.append((item[0], item[1], item[2], 255))

    img.putdata(newData)
    img.save(dst_path, "PNG")
    
def paste_image(path_first_image, path_second_image, merged_path):
    """
    Overlays second image on top of the first one
    Arguments:
        path_first_image: path to first image
        path_second_image: path to second image
        merged_path: path to save merged image
    """
    background = Image.open(path_first_image)
    foreground = Image.open(path_second_image)
    background = background.convert("RGBA")
    foreground = foreground.convert("RGBA")
    
    background.paste(foreground, (0, 0), foreground)
    
    background = background.convert("RGBA")
    background.save(merged_path, "PNG")

def create_dir(path):
    """
    Creates a directory if it doesn't exist
    Arguments:
        path: path to the directory
    """
    if not os.path.exists(path):
        os.makedirs(path)


def read_image(path):
    """
    reads image in color mode and resizes to default size
    Arguments:
        path: path to the image
    Returns:
        original image in color, resized image with expanded dimension
    """
    x = cv2.imread(path, cv2.IMREAD_COLOR) 
    x = cv2.resize(x, (W, H))
    ori_x = x
    x = x/255.0
    x = x.astype(np.float32)
    x = np.expand_dims(x, axis=0)
    return ori_x, x                                

def read_mask(path):
    """
    reads image in grayscale mode and resizes to default size
    Arguments:
        path: path to the image
    Returns:
        original image in grayscale, resized image with expanded dimension
    """
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE) 
    x = cv2.resize(x, (W, H))
    ori_x = x
    x = x/255.0
    x = x.astype(np.int32)                
    return ori_x, x

def make_collage(image1, image2, image3, save_image_path):
    """
    Makes an image collage by horizontally conacatenating three images
    Arguments:
        image1: first image
        image2: second image
        image3: third image
        save_image_path : path to save the collage
    """
    line = np.ones((H, 10, 3)) * 255

    image2 = np.expand_dims(image2, axis=-1)
    image2 = np.concatenate([image2, image2, image2], axis=-1)

    cat_image = np.concatenate([image1, line, image2, line, image3], axis=1)
    cv2.imwrite(save_image_path, cat_image)

In [4]:
#create required directories
create_dir(binarized_folder)
create_dir(overlay_folder)
create_dir(collage_folder)
create_dir(transparent_folder)

In [5]:
for image_name in sorted(os.listdir(dermoscopic_image_folder)):
    dermoscopic_image_path =  os.path.join(base_dir,dermoscopic_image_folder, image_name)
    mask_path = os.path.join(base_dir,binarized_folder, image_name)
    transparent_mask_path = os.path.join(base_dir,transparent_folder, image_name)
    overlay_image_path = os.path.join(base_dir,overlay_folder, image_name)
    collage_image_path = os.path.join(base_dir,collage_folder, image_name)
    
    #binarize mask image
    mask_path = os.path.join(base_dir,hair_mask_folder, image_name)
    im = cv2.imread(mask_path)
    binarized_image_path = os.path.join(binarized_folder, image_name)
    th, im_th = cv2.threshold(im, 50, 255, cv2.THRESH_BINARY)
    cv2.imwrite(binarized_image_path, im_th)
    
    #make binarized mask transparent
    make_transparent(mask_path, transparent_mask_path)
    
    #overlay transparent mask on dermoscopic image
    paste_image(dermoscopic_image_path, transparent_mask_path, overlay_image_path)
    
    #create collage
    ori_dermo, x = read_image(dermoscopic_image_path)
    ori_mask, y = read_mask(mask_path)
    ori_overlay, z = read_image(overlay_image_path)
    make_collage(ori_dermo, ori_mask, ori_overlay, collage_image_path)